In [4]:
import numpy as np
from keras.models import load_model

In [10]:
import string

In [5]:
def read_glove_vecs(glove_file):
    with open(glove_file,encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype='float32')
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove6b200d/glove.6B.200d.txt')

In [8]:
model = load_model('chatter.h5')

W1123 03:22:20.983258 21800 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1123 03:22:21.529695 21800 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1123 03:22:21.529695 21800 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1123 03:22:22.568818 21800 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1123 03:22:22.576824 21800 de

In [14]:
table = str.maketrans('', '', string.punctuation)
def pp(strr):
    words = strr.split()
    stripped = [w.translate(table) for w in words]
    #stripped = list(filter(None, stripped))
    stripped = [word.lower() for word in stripped]
    #stripped = [w for w in stripped if not w in stop_words]
    #stemmed = [porter.stem(word) for word in stripped]
    return(' '.join(stripped))

In [16]:
def sentences_to_indices(X, word_to_index, max_len):

    m = X.shape[0]                                   # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        sentence_words = X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+ 1
            
 
    return X_indices

In [17]:
def chat():
    print("start chit chat! (Enter quit to exit)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        x_test = np.array([pp(inp)])
        X_test_indices = sentences_to_indices(x_test, word_to_index, 24)
        pred = model.predict(X_test_indices)
        num = np.argmax(pred)
        print(num)

In [19]:
chat()

start chit chat! (Enter quit to exit)
You: how much do i need to pay
2
You: quit


In [30]:
test = "this is a test sentence this can be more than 24 lines i hope, lets check it out, hopefully the model can take sentence"

In [32]:
roll = sentences_to_indices(np.array([pp(test)]), word_to_index, 24)
print(roll)

[[358160. 192973.  43010. 356531. 325080. 358160.  90548.  71090. 249714.
  357109.  19410. 222999. 185457. 181581. 220930.  97762. 193716. 272930.
  181587. 357266. 247378.  90548. 351935. 325080.]]


In [33]:
model.predict(roll)

array([[5.4455856e-03, 2.7111453e-01, 2.7956029e-02, 1.3245712e-02,
        4.9426073e-01, 1.2552427e-01, 5.9306845e-02, 8.1191928e-04,
        1.0349957e-03, 1.0124220e-03, 2.8699625e-04]], dtype=float32)

In [37]:
import json
import nltk

In [35]:
with open("intents.json") as file:
    data = json.load(file)

In [50]:
words = []
labels = []
docs_x = []
docs_y = []
xx = []
for intent in data["intents"]:
        for pattern in intent["patterns"]:
            ##stemming to eliminate extra words
            ##tokeninze
            wrds = nltk.word_tokenize(pattern) ##list of words in patters
            words.extend(wrds) ##instead of append
            xx.append(pattern)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
            
labels = sorted(labels)      

In [53]:
fasa=[]
for line in xx:
    words = line.split()
    stripped = [w.translate(table) for w in words]
    #stripped = list(filter(None, stripped))
    stripped = [word.lower() for word in stripped]
    #stripped = [w for w in stripped if not w in stop_words]
    #stemmed = [porter.stem(word) for word in stripped]
    fasa.append(' '.join(stripped))
X_train = [i for i in fasa if i]

In [56]:
len(X_train)==len(docs_y)

True

In [87]:
dictt = {}
for i,n in enumerate(X_train):
    if not docs_y[i] in dictt.keys():
        dictt[docs_y[i]]=[]
    dictt[docs_y[i]].append(sentences_to_indices(np.array([pp(n)]), word_to_index, 24))

In [97]:
len(dictt["greeting"])

5

<h1>TO DO: </h1>
<h2>save the dictionary of wordstovectors while training and also save the lookup list for the tags to look for the output</h2>
<h2>add comments </h2>
<h2>add cosine similarity function</h2>